# Training notebook

Here we tune parameters of models. From the 7 available seasons, we use first 4 for training and the rest 3 go in testing. There is no fitting on data as such and so we just take the average performance over 4 training seasons to see which parameters work best.

In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
np.random.seed(1234)

import sys
sys.path.append("../src")
from utils import data as udata
from utils import dists as udists
from utils import misc as u
from truth import mask_truths
from predictors import make_predictor
import losses
from pymmwr import Epiweek
from tqdm import tqdm, trange
import ledge.merge as merge
import ledge.update as update
import ledge.fill as fill
from functools import partial
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import hyperopt.pyll.stochastic as st
import json
import os.path as path
import inspect

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
EXP_DIR = "../data/processed/cdc-flusight-ensemble/"
OUTPUT_DIR = "../models/cdc-flusight-ensemble/"
TARGET = "4-ahead"
MAX_LAG = 29
# REGIONS = ["nat", *[f"hhs{i}" for i in range(1, 11)]]
REGIONS = ["nat"] # Speed up!
TRAINING_SEASONS = list(range(2010, 2014))
LOSS_FN = losses.ploss

In [3]:
components = [udata.Component(EXP_DIR, m) for m in udata.available_models(EXP_DIR)]
ad = udata.ActualData(EXP_DIR)

# Evaluation

In [4]:
def evaluate(predictor, loss_fn=losses.ploss):
    """
    Evaluate the predictor over all training seasons and regions and return mean score
    """
    
    first_losses = []
    final_losses = []
    
    with tqdm(total=len(TRAINING_SEASONS) * len(REGIONS)) as pbar:
        for season in TRAINING_SEASONS:
            for region in REGIONS:
                truths = [ad.get(TARGET, region, season, lag=l) for l in range(MAX_LAG + 1)]
                c_preds = [cmp.get(TARGET, region, season) for cmp in components]
            
                first_truth = merge.earliest(truths)
                final_truth = merge.latest(truths)
                pred = predictor(truths, c_preds)
                first_losses.append(float(loss_fn(pred, first_truth).mean()))
                final_losses.append(float(loss_fn(pred, final_truth).mean()))
                pbar.update()

    return {
        "first_loss": np.mean(first_losses),
        "final_loss": np.mean(final_losses)
    }

# Tuning

In [5]:
# Merge strategy
MERGE_FN = merge.latest

## Search space for imputation

In [8]:
fill_space = hp.choice("imputation_type", [
    { "type": None },
    {
        "type": "diff",
        "lookback": 1 + hp.randint("lookback", 33),
        "normalize": hp.choice("normalize", [True, False]),
        "window": hp.choice("window", [
            {
                "type": "uniform"
            },
            {
                "type": "linear",
                "alpha": hp.uniform("alpha", 0, 1)
            },
            {
                "type": "geometric",
                "gamma": hp.uniform("gamma", 0, 1)
            }
        ]),
        "incremental": hp.choice("incremental", [True, False]),
        "n_series": 2 + hp.randint("n_series", 30)
    }
])

In [27]:
def impute(series_list, config):
    if len(series_list) < 2:
        return series_list
    if config["type"] is None:
        return series_list
    elif config["type"] is "diff":
        window = config["window"]
        if window["type"] == "uniform":
            window_fn = fill.window_uniform
        elif window["type"] == "linear":
            window_fn = partial(fill.window_linear, alpha=window["alpha"])
        elif window["type"] == "geometric":
            window_fn = partial(fill.window_geometric, gamma=window["gamma"])
        else:
            raise Exception("Window type not understood")
        
        window_fn = fill.lookback(config["lookback"])(window_fn)
        if config["normalize"]:
            window_fn = fill.normalize(window_fn)
            
        series_list[:config["n_series"]] = fill.diff_window(
            series_list[:config["n_series"]],
            window_fn,
            inc=config["incremental"]
        )
        return series_list
    else:
        raise Exception("Imputation type not understood")

## Individuals

In [6]:
for idx, cmp in enumerate(components):
    update_fn = partial(update.pick, index=idx)
    losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update_fn), LOSS_FN)
    print(f"{cmp.name}: {losses['first_loss'], losses['final_loss']}")

  0%|          | 0/4 [00:00<?, ?it/s]

CU-BMA: (0.9516458082462813, 0.9455271895144688)


  0%|          | 0/4 [00:00<?, ?it/s]

CU-EAKFC_SEIRS: (0.9480731224971222, 0.9261226093092367)


  0%|          | 0/4 [00:00<?, ?it/s]

...

ReichLab-SARIMA1: (0.9316378876486374, 0.9375369426425701)


  0%|          | 0/4 [00:00<?, ?it/s]

ReichLab-SARIMA2: (0.9299500088607586, 0.9346892153698427)


100%|██████████| 4/4 [00:18<00:00,  4.78s/it]

UTAustin-edm: (0.9394295543153042, 0.9517505790062064)


## Mean

In [28]:
trials = Trials()
best = fmin(hmin, space=fill_space, algo=tpe.suggest, max_evals=100, trials=trials)
print(best)

100%|██████████| 4/4 [00:21<00:00,  5.31s/it]

{'imputation_type': 1, 'incremental': 0, 'lookback': 22, 'n_series': 3, 'normalize': 0, 'window': 0}


In [42]:
trials.best_trial["result"]

{'config': {'incremental': True,
  'lookback': 23,
  'n_series': 5,
  'normalize': True,
  'type': 'diff',
  'window': {'type': 'uniform'}},
 'first_loss': 0.8922135537419298,
 'loss': 0.9024205112357531,
 'status': 'ok'}

## Dem

In [8]:
# Load the weights from DEM training done in tracking-ensemble repository
with open(path.join(OUTPUT_DIR, TARGET, "dem-weight-ensemble.json")) as fp:
    w_vec = json.load(fp)["fit_params"]["weights"]
    models = [cmp.name for cmp in components]
    weights = xr.DataArray(w_vec, dims="model", coords={ "model": models })
    
update_fn = partial(update.noop, init_weights=weights)
losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update_fn), LOSS_FN)
print(f"{losses['first_loss'], losses['final_loss']}")

100%|██████████| 4/4 [00:16<00:00,  4.09s/it]

(0.91701711495345, 0.9185189211648521)


## Follow the leader

In [9]:
def hmin(fill_config):
    FILL_FN = partial(impute, config=fill_config)
    losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update.ftl, fill_fn=FILL_FN),
                      LOSS_FN)
    return {
        "loss": losses["final_loss"],
        "status": STATUS_OK,
        "first_loss": losses["first_loss"],
        "config": fill_config
    }

trials = Trials()
best = fmin(hmin, space=fill_space, algo=tpe.suggest, max_evals=100, trials=trials)
print(trials.best_trial["result"])

100%|██████████| 4/4 [00:16<00:00,  4.15s/it]

(0.887509003361017, 0.903981443179864)


## MW

In [10]:
for eta in np.linspace(0.5, 1.0, 11):
    update_fn = partial(update.mw, eta=eta)
    losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update_fn), LOSS_FN)
    print(f"{eta}: {losses['first_loss'], losses['final_loss']}")

  0%|          | 0/4 [00:00<?, ?it/s]

0.5: (0.9282935726604933, 0.9269872566873232)


  0%|          | 0/4 [00:00<?, ?it/s]

0.55: (0.926748332159582, 0.9253980323816449)


  0%|          | 0/4 [00:00<?, ?it/s]

...

0.9: (0.9099636900914446, 0.9145738709610577)


  0%|          | 0/4 [00:00<?, ?it/s]

0.95: (0.9074302796489346, 0.915860868662467)


100%|██████████| 4/4 [00:22<00:00,  5.58s/it]

1.0: (0.9149758089626523, 0.9219862242655469)


## Hedge

In [19]:
for eta in np.linspace(0.5, 30.0, 11):
    update_fn = partial(update.hedge, eta=eta)
    losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update_fn), LOSS_FN)
    print(f"{eta}: {losses['first_loss'], losses['final_loss']}")

  0%|          | 0/4 [00:00<?, ?it/s]

0.5: (0.9320335841461582, 0.9309752144307468)


  0%|          | 0/4 [00:00<?, ?it/s]

3.45: (0.9111900429099099, 0.9140675746004779)


  0%|          | 0/4 [00:00<?, ?it/s]

...

24.1: (0.8903006563592988, 0.9070814045453076)


  0%|          | 0/4 [00:00<?, ?it/s]

27.05: (0.8965621010055009, 0.9168197372746651)


100%|██████████| 4/4 [00:17<00:00,  4.50s/it]

30.0: (0.9240332679780662, 0.9404880677641445)


## Fixed share

In [20]:
for alpha in np.linspace(0, 0.5, 6):
    for eta in np.linspace(0.5, 30.0, 11):
        update_fn = partial(update.fixed_share, eta=eta, alpha=alpha)
        losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update_fn), LOSS_FN)
        print(f"{eta}, {alpha}: {losses['first_loss'], losses['final_loss']}")

  0%|          | 0/4 [00:00<?, ?it/s]

0.5, 0.0: (0.9320335841461582, 0.9309752144307468)


  0%|          | 0/4 [00:00<?, ?it/s]

3.45, 0.0: (0.9111900429099099, 0.9140675746004779)


  0%|          | 0/4 [00:00<?, ?it/s]

...

24.1, 0.5: (0.9148254398261768, 0.9174323986391406)


  0%|          | 0/4 [00:00<?, ?it/s]

27.05, 0.5: (0.9155744103108464, 0.9180728827577083)


100%|██████████| 4/4 [00:59<00:00, 14.88s/it]

30.0, 0.5: (0.9350052950000417, 0.9373201610779494)
